In [77]:
import re
import sys
import traceback
import shutil
import codecs
import warnings 
warnings.filterwarnings ('ignore')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 7]

import itertools
from decimal import Decimal
import datetime 
from datetime import datetime
import string
def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

import json
import requests
from pyquery import PyQuery as pq


from cocoNLP.extractor import extractor

# from selenium import webdriver 
# from selenium.webdriver import ChromeOptions

import folium
print(folium.__version__)

# amap
url_amap = 'http://restapi.amap.com/v3/geocode/geo?parameters'
KEY_amap = '460c7053ffe6fd4767354484f72f30df' 

# weibo
username_wb='wangjl_1995@163.com'
password_wb='wjl19950324'
App_Key：'1394316129'
App_Secret：'ababc16e7f47dba10d46bd4c7ae5c688'

0.12.1.post1


In [197]:
url_wb = 'https://m.weibo.cn/api/container/getIndex?'
 
headers ={
    'Host':'m.weibo.cn',
    'Referer': 'https://m.weibo.cn/u/1644948230',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'}

query = {'type':'uid',
         'value':'1644948230'}

def get_containerid():
    query = {'type':'uid',
         'value':'1644948230'}
    url = url_wb + urlencode(query)
    try:
        response = requests.get(url,headers=headers)
        
        if response.status_code == 200:
            containerid_dict={}
            wb_fp=response.json()
            
            for item in wb_fp['data']['tabsInfo']['tabs']:
                containerid_dict[item['tabKey']]=item['containerid']
            return containerid_dict
        
    except requests.ConnectionError as e:
        print('Get First Page Error',e.args)

# containerid=get_containerid()['weibo']
start_date='2022-01-15' #'2021-12-01'

def readHtml(start_date):

    query['containerid']='1076031644948230'
    query['since_id']=''
        
    page = 1 
    
    info_list=[]
    info_col=[ 'mid', 'time','reposts_count', 'comments_count', 'attitudes_count','title','content1','text']
    
    start_dt=datetime.strptime(start_date+' 00:00:00 +0800','%Y-%m-%d %H:%M:%S %z')
    start_str=datetime.strftime(start_dt, "%Y-%m-%d_%H:%M:%S")
    
    while page:    

        url = url_wb + urlencode(query)
        response = requests.get(url,headers=headers,verify=False)
#         if response.status_code == 200:
        index_json=response.json()
#         print(index_json)
        cards = index_json['data']['cards']
        query['since_id']=index_json['data']['cardlistInfo']['since_id']
        cnt=1
        max_time=datetime.strptime(cards[0]['mblog']['created_at'],'%a %b %d %H:%M:%S %z %Y')
        
        if max_time>=start_dt:
            print('current maximum time: ',max_time, ' start reading......................')
            
            for card in cards:
                try:
                    mblog=card['mblog']
                    if mblog['page_info']['type']=='search_topic':
                        mid = mblog['mid']
                        time=datetime.strptime(mblog['created_at'],'%a %b %d %H:%M:%S %z %Y')
                        reposts_count = mblog['reposts_count']
                        comments_count = mblog['comments_count']
                        attitudes_count = mblog['attitudes_count']
                        title=mblog['page_info']['page_title']
                        content1=mblog['page_info']['content1']
                        text = pq(mblog['text']).text()
                        isLongText = mblog['isLongText']
                        if isLongText:
                            text_id = mblog['id']
                            text_href = 'http://m.weibo.cn/statuses/extend?id=%s'%text_id
                            text_res = requests.get(text_href,headers=headers,verify=False)
                            text_json=text_res.json()
                            text = pq(text_json['data']['longTextContent']).text()

                        info = (mid, time, reposts_count, comments_count, attitudes_count,title,content1,text)
                        info_list.append(info)
                        print('page ',page,' card ',cnt,' inserted')
                        
                    else:
                        print('page ',page,' card ',cnt,' omitted')
                    
                    cnt+=1

                except Exception as e:
                    traceback.print_exc()
                    continue

            print ('get page:%d'%page)
            page += 1
            
        else:
            print('current maximum time: ',max_time, ', time earlier than start date: ',start_dt,)
            break

    info_df = pd.DataFrame(info_list,columns=info_col)
    info_df = info_df.astype(str)
    
    info_df['time']=info_df['time'].str[:19]
    
    ts=datetime.strftime(time.today(), "%Y-%m-%d_%H:%M:%S")
    
    info_df.to_excel('xinguan%s_%s_to_%s.xlsx'%(query['value'],start_str,ts), index=False)
    
    return info_df
#     except Exception as e:
#         traceback.print_exc()

In [199]:
adf=readHtml(start_date)

current maximum time:  2022-01-26 10:31:40+08:00  start reading......................
page  1  card  1  omitted
page  1  card  2  omitted
page  1  card  3  omitted
page  1  card  4  omitted
page  1  card  5  inserted
page  1  card  6  omitted
page  1  card  7  omitted
page  1  card  8  omitted
page  1  card  9  omitted
get page:1


Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/2680871037.py", line 64, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


current maximum time:  2022-01-25 22:21:17+08:00  start reading......................
page  2  card  1  omitted
page  2  card  2  inserted
page  2  card  3  omitted
page  2  card  4  omitted
page  2  card  5  omitted
page  2  card  6  omitted
page  2  card  7  omitted
page  2  card  8  inserted
page  2  card  9  omitted
page  2  card  10  omitted
get page:2
current maximum time:  2022-01-25 19:00:02+08:00  start reading......................
page  3  card  1  omitted
page  3  card  2  inserted
page  3  card  3  omitted
page  3  card  4  inserted
page  3  card  5  omitted
page  3  card  6  omitted
page  3  card  7  omitted
page  3  card  8  omitted
page  3  card  9  omitted
page  3  card  10  inserted
get page:3
current maximum time:  2022-01-25 17:31:38+08:00  start reading......................
page  4  card  1  inserted
page  4  card  2  inserted
page  4  card  3  omitted
page  4  card  4  omitted
page  4  card  5  inserted
get page:4


Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/2680871037.py", line 64, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'
Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/2680871037.py", line 64, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


current maximum time:  2022-01-25 16:53:13+08:00  start reading......................
page  5  card  1  inserted
page  5  card  2  inserted
page  5  card  3  inserted
page  5  card  4  omitted
page  5  card  5  omitted
page  5  card  6  inserted
page  5  card  7  inserted


...

current maximum time:  2022-01-17 12:02:37+08:00  start reading......................
page  57  card  1  inserted
page  57  card  2  inserted
page  57  card  3  inserted
page  57  card  4  inserted
page  57  card  5  inserted
page  57  card  6  inserted


Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/2680871037.py", line 64, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


page  57  card  7  inserted
page  57  card  8  inserted
page  57  card  9  omitted
get page:57
current maximum time:  2022-01-17 10:55:11+08:00  start reading......................
page  58  card  1  omitted
page  58  card  2  omitted
page  58  card  3  inserted
page  58  card  4  inserted
page  58  card  5  inserted
page  58  card  6  inserted
page  58  card  7  omitted
page  58  card  8  omitted
page  58  card  9  inserted
get page:58
current maximum time:  2022-01-17 08:32:01+08:00  start reading......................
page  59  card  1  omitted
page  59  card  2  inserted
page  59  card  3  omitted
page  59  card  4  omitted
page  59  card  5  omitted
page  59  card  6  omitted
page  59  card  7  omitted
page  59  card  8  omitted
page  59  card  9  omitted
page  59  card  10  omitted
get page:59
current maximum time:  2022-01-16 20:30:03+08:00  start reading......................
page  60  card  1  omitted
page  60  card  2  omitted
page  60  card  3  omitted
page  60  card  4  omi

Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/2680871037.py", line 64, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


current maximum time:  2022-01-15 15:10:00+08:00  start reading......................
page  68  card  1  inserted
page  68  card  2  omitted
page  68  card  3  omitted
page  68  card  4  inserted
page  68  card  5  omitted
page  68  card  6  inserted
page  68  card  7  inserted
page  68  card  8  omitted
page  68  card  9  omitted
page  68  card  10  omitted
get page:68
current maximum time:  2022-01-15 11:40:34+08:00  start reading......................
page  69  card  1  omitted
page  69  card  2  omitted
page  69  card  3  omitted
page  69  card  4  omitted
page  69  card  5  omitted
page  69  card  6  omitted
page  69  card  7  inserted
page  69  card  8  omitted
page  69  card  9  omitted
page  69  card  10  omitted
get page:69
current maximum time:  2022-01-14 22:48:23+08:00 , time earlier than start date:  2022-01-15 00:00:00+08:00


In [ ]:
ex = extractor()

def location_from_text(text):
    A=[]
    word_filter=['核酸','全域','从业者','职员','某','卫健委','人员','工作人员','报告','冬奥','病例','区长','时间']
    district_list=['东城','西城','朝阳','丰台','石景山','海淀','门头沟','房山','通州','顺义','昌平','大兴','怀柔','平谷','密云','延庆']
    for loc in ex.extract_locations(text):
        if (len(loc)>=5) and (any(ele in loc for ele in location_list)==True) \
        and (any(ele in loc for ele in word_filter)==False) and (loc not in A) : 
            for district in district_list:
                if district in loc and '区' not in loc:
                    loc=loc.replace(district,district+'区')
                loc=loc.replace('北京市','').replace('北京','')
            A.append(loc)
    return [x for x in A if not any(x in y and x!=y for y in A)]

In [389]:
adf['xinguan_flag']= adf['text'].apply(lambda x: any(ele in x for ele in xinguan_list))
adf['xinguan_dtl']= adf['text'].apply(lambda x: [ele in x for ele in xinguan_list])

# adf['beijing_flag'] = adf['text'].apply(lambda x: any(ele in x for ele in beijing_list))

adf['location_flag'] = adf['text'].apply(lambda x: any(ele in x for ele in location_list))
adf['location_dtl']= adf['text'].apply(lambda x: location_from_text(x))

adf['flag_count']=adf['xinguan_flag'].astype(int) + adf['location_flag'].astype(int) 

In [391]:
bdf=adf[adf.flag_count==2]
bdf

,mid,time,reposts_count,comments_count,attitudes_count,title,content1,text,beijing_flag,xinguan_flag,location_flag,flag_count,xinguan_dtl,location_dtl
0,4729812704824673,2022-01-26 08:58:07,0,0,1,#1月25日新增本土确诊24例#,0讨论 0阅读,【#1月25日新增本土确诊24例#，其中#北京新增14例本土确诊#】1月25日0—24时，3...,True,True,True,2,"[True, False, False, True, False, True, False,...",[]
2,4729614260502710,2022-01-25 19:49:33,0,6,10,#涞水县1例确诊系北京务工返乡人员#,0讨论 0阅读,【河北保定#涞水县1例确诊系北京务工返乡人员#】保定市涞水县应对新冠肺炎疫情工作领导小组办公...,True,True,True,2,"[False, True, True, False, True, False, True, ...","[丰台区, 房山区张坊镇]"
5,4729580922867770,2022-01-25 17:37:06,24,81,309,#丰台区新增8例新冠感染者详细轨迹#,0讨论 0阅读,【#丰台区新增8例新冠感染者详细轨迹# 】1月25日，北京市第275场新冠肺炎疫情防控工作新...,True,True,True,2,"[True, True, True, True, False, True, True, Fa...","[丰台区郑王坟, 丰台区丰桥路]"
8,4729570060407485,2022-01-25 16:53:56,3,0,4,#西城区两地划为封控区#,0讨论 0阅读,#西城区两地划为封控区#【#西城区广外鸭子桥路1号院2号楼被划定为封控区#】1月25日，北京...,True,True,True,2,"[False, True, False, True, False, False, True,...",[西城区广外鸭子桥路]
9,4729569879526021,2022-01-25 16:53:13,3,5,13,#西城3例阳性感染者详情发布#,0讨论 0阅读,#西城3例阳性感染者详情发布#【#西城区检测出3例核酸阳性感染者详情#发布】1月25日，北京...,True,True,True,2,"[False, True, False, True, True, False, True, ...","[西城区右安门西街甲, 玉泉营桥西]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,4725999637366244,2022-01-15 20:26:21,0,1,1,#北京海淀所涉17个风险点均已管控#,0讨论 0阅读,【北京海淀新增1例确诊病例 #北京海淀所涉17个风险点均已管控# 】1月15日晚20时许，#...,True,True,True,2,"[True, True, False, False, False, False, True,...","[海淀区上地街道农大南路, 海淀区街道海淀区新技术大厦]"
241,4725998648559421,2022-01-15 20:22:25,0,3,3,#北京新增确诊为奥密克戎#,0讨论 0阅读,【北京海淀新增1例确诊病例 #北京新增确诊为奥密克戎# 】1月15日晚20时许，#北京市召开...,True,True,True,2,"[True, True, False, False, False, False, False...","[海淀区上地街道农大南路, 海淀区街道海淀区新技术大厦]"
242,4725998362300105,2022-01-15 20:21:17,4,8,2,#北京市召开第266场疫情防控新闻发布会#,0讨论 0阅读,【北京海淀新增1例确诊病例 工作单位在海淀街道海淀新技术大厦】1月15日晚20时许，#北京市...,True,True,True,2,"[True, True, False, False, False, False, False...","[海淀区街道海淀区新技术大厦, 海淀区上地街道农大南路, 海淀区街道海淀区新技术大厦]"
245,4725997214110530,2022-01-15 20:16:43,213,446,1231,#北京今日新增1例新冠肺炎确诊病例#,0讨论 0阅读,【#北京今日新增1例新冠肺炎确诊病例# 】1月15日，北京市海淀区报告一名新冠病毒核酸阳性人...,True,True,True,2,"[True, True, False, False, True, False, True, ...",[]


In [393]:
# unique addresses
flat = list(set(itertools.chain.from_iterable(bdf['location_dtl'])))
addr_all = sorted([x for x in flat if not any(x in y and x!=y for y in flat)])
addr_all

['中关村国际',
 '丰台区两地',
 '丰台区丰桥路',
 '丰台区华兴饭庄',
 '丰台区南庭',
 '丰台区南苑街道',
 '丰台区右安门街道菜户营东街',
 '丰台区宜兰园',
 '丰台区怡海花园恒丰园',
 '丰台区新村街道丰桥路',
 '丰台区新村街道怡海花园恒丰园小区',
 '丰台区玉泉营果品公司',
 '丰台区玉泉营果品冷库',
 '丰台区玉泉营街道云岭商务楼',
 '丰台区玉泉营街道京丰公寓南院',
 '丰台区玉泉营街道宜兰园',
 '丰台区玉泉营街道玉泉营',
 '丰台区玉泉营街道郑王坟',
 '丰台区玉泉营街道马家楼村',
 '丰台区石榴园西街',
 '丰台区禽蛋公司',
 '丰台区蒲黄榆第二社区',
 '丰台区西南郊冷库三新公司',
 '丰台区郑王坟',
 '丰台区马家楼',
 '丰台区黄土岗马',
 '大兴区安定镇',
 '大兴区旧宫镇庑殿路祥龙大院',
 '大兴区旧宫镇美利新世界',
 '房山区张坊镇',
 '朝阳区大屯路甲',
 '朝阳区安贞街道安贞里',
 '朝阳区平房乡石各庄村',
 '海淀区上地信息路',
 '海淀区上地街道农大南路',
 '海淀区专业部门',
 '海淀区中关村末端站点',
 '海淀区丹棱街',
 '海淀区农科社区',
 '海淀区本土',
 '海淀区桥北至厢黄旗',
 '海淀区永丰路百旺茉莉园',
 '海淀区街道海淀区新技术大厦',
 '海淀区西三旗建材城东路',
 '海淀区西街十字路口',
 '海淀区马连洼街道永丰路百旺茉莉园',
 '海淀朝阳丰台区房山',
 '清河万象',
 '清河小营西站',
 '玉泉营便利店',
 '玉泉营南路西南郊冷库',
 '玉泉营支行',
 '玉泉营果品公司冷库',
 '玉泉营果品公司宿舍',
 '玉泉营桥西',
 '玉泉营街道纪',
 '经济开发区博兴街道亦庄',
 '西城区右安门西街甲',
 '西城区广外鸭子桥路',
 '西城区牛街',
 '西城区白纸坊街道右安门西街甲',
 '西城区陶然亭街道',
 '通州区西集镇',
 '金融街购物中心闭店',
 '长阳百花峰',
 '长阳碧波园店',
 '长阳绿地万民百汇',
 '长阳镇万兴路',
 '长阳镇北广阳城大街']

In [379]:
beijing_list=['北京','东城', '东华门', '景山', '交道口', '安定门', '北新桥', '东四', '朝阳门', '建国门', '东直门', '和平里', '前门', '崇文门外', '东花市', '龙潭', '体育馆路', '天坛', '永定门外',                          
'西城', '西长安街', '新街口', '月坛', '展览路', '德胜', '金融街', '什刹海', '大栅栏', '天桥', '椿树', '陶然亭', '广内', '牛街', '白纸坊', '广外',                            
'朝阳', '建外', '朝外', '呼家楼', '三里屯', '左家庄', '香河园', '和平街', '安贞', '亚运村', '小关', '酒仙桥', '麦子店', '团结湖', '六里屯', '八里庄', '双井', '劲松', '潘家园', '垡头', '南磨房', '高碑店', '将台', '太阳宫', '大屯', '望京', '小红门', '十八里店', '平房', '东风', '奥运村', '来广营', '常营', '三间房', '管庄', '金盏', '孙河', '崔各庄', '东坝', '黑庄户', '豆各庄', '王四营', '东湖', '首都机场',
'丰台', '右安门', '太平桥', '西罗园', '大红门', '南苑', '东高地', '东铁匠营六里桥', '丰台', '新村', '长辛店', '云岗', '方庄', '宛平', '马家堡和义', '卢沟桥', '花乡', '玉泉营', '成寿寺', '看丹', '五里店', '青塔', '石榴庄', '北宫', '王佐',                   
'石景山', '八宝山', '老山', '八角', '古城', '苹果园', '金顶街', '广宁', '五里坨', '鲁谷',                                  
'海淀', '万寿路', '永定路', '羊坊店', '甘家口', '八里庄', '紫竹院', '北下关北太平庄', '学院路', '中关村', '海淀', '青龙桥', '清华园', '燕园', '香山', '清河', '花园路', '西三旗', '马连洼', '田村路', '上地', '万柳', '曙光', '温泉', '四季青', '西北旺', '苏家坨', '上庄', '东升',               
'门头沟', '大峪', '城子', '东辛房', '大台', '王平', '永定', '龙泉', '潭柘寺', '军庄', '雁翅', '斋堂', '清水', '妙峰山',                              
'房山', '城关', '新镇', '向阳', '东风', '迎风', '星城', '良乡', '周口店', '琉璃河', '拱辰', '西潞', '阎村', '窦店', '石楼', '长阳', '长沟', '大石窝', '张坊', '十渡', '青龙湖', '韩村河', '霞云岭', '南窖', '佛子庄', '大安山', '史家营', '蒲洼',               
'通州', '中仓', '新华', '北苑', '玉桥', '潞源', '文景', '九棵树', '临河里', '杨庄', '潞邑', '永顺', '梨园', '宋庄', '张家湾', '漷县', '马驹桥', '西集', '台湖', '永乐店', '潞城', '于家务回族',                      
'顺义', '胜利', '光明', '仁和', '后沙峪', '天竺', '杨镇', '牛栏山', '南法信马坡', '石园', '空港', '双丰', '旺泉', '高丽营', '李桥', '李遂', '南彩', '北务', '大孙各庄', '张镇', '龙湾屯', '木林', '北小营', '北石槽', '赵全营',                   
'昌平', '城北', '南口', '马池口', '沙河', '城南', '东小口', '天通苑北', '天通苑南', '霍营', '回龙观', '龙泽园', '史各庄', '阳坊', '小汤山', '南邵', '崔村百善', '北七家', '兴寿', '流村', '十三陵', '延寿',                      
'大兴', '兴丰', '林校路', '清源', '亦庄', '黄村', '旧宫', '西红门', '瀛海', '观音寺', '天宫院', '高米店', '荣华', '博兴', '青云店', '采育', '安定', '礼贤榆垡', '庞各庄', '北臧村', '魏善庄', '长子营',                      
'怀柔', '泉河', '龙山', '怀柔', '雁栖', '庙城', '北房', '杨宋', '桥梓', '怀北', '汤河口', '渤海', '九渡河', '琉璃庙', '宝山', '长哨营', '喇叭沟门',                           
'平谷', '滨河', '兴谷', '渔阳', '峪口', '马坊', '金海湖', '东高村', '山东庄', '南独乐河', '大华山', '夏各庄', '马昌营', '王辛庄', '大兴庄', '刘家店', '罗营', '黄松峪熊儿寨',                          
'密云', '鼓楼', '果园', '檀营', '密云', '溪翁庄', '西田各庄', '十里堡', '河南寨', '巨各庄', '穆家峪', '太师屯', '高岭', '不老屯', '冯家峪', '古北口', '大城子', '东邵渠', '北庄新城子', '石城',                        
'延庆', '百泉', '香水园', '儒林', '延庆', '康庄', '八达岭', '永宁', '旧县', '张山营', '四海', '千家店', '沈家营', '大榆树', '井庄', '大庄科', '刘斌堡', '香营', '珍珠泉']
location_list=['东华门', '景山', '交道口', '安定门', '北新桥', '东四', '朝阳门', '建国门', '东直门', '和平里', '前门', '崇文门外', '东花市', '龙潭', '体育馆路', '天坛', '永定门外',                                              
'西长安街', '新街口', '月坛', '展览路', '德胜', '金融街', '什刹海', '大栅栏', '天桥', '椿树', '陶然亭', '广内', '牛街', '白纸坊', '广外',                                                
'建外', '朝外', '呼家楼', '三里屯', '左家庄', '香河园', '和平街', '安贞', '亚运村', '小关', '酒仙桥', '麦子店', '团结湖', '六里屯', '八里庄', '双井', '劲松', '潘家园', '垡头', '南磨房', '高碑店', '将台', '太阳宫', '大屯', '望京', '小红门', '十八里店', '平房', '东风', '奥运村', '来广营', '常营', '三间房', '管庄', '金盏', '孙河', '崔各庄', '东坝', '黑庄户', '豆各庄', '王四营', '东湖', '首都机场',                    
'右安门', '太平桥', '西罗园', '大红门', '南苑', '东高地', '东铁匠营六里桥', '丰台', '新村', '长辛店', '云岗', '方庄', '宛平', '马家堡和义', '卢沟桥', '花乡', '玉泉营', '成寿寺', '看丹', '五里店', '青塔', '石榴庄', '北宫', '王佐',                                       
'八宝山', '老山', '八角', '古城', '苹果园', '金顶街', '广宁', '五里坨', '鲁谷',                                                      
'万寿路', '永定路', '羊坊店', '甘家口', '八里庄', '紫竹院', '北下关北太平庄', '学院路', '中关村', '海淀', '青龙桥', '清华园', '燕园', '香山', '清河', '花园路', '西三旗', '马连洼', '田村路', '上地', '万柳', '曙光', '温泉', '四季青', '西北旺', '苏家坨', '上庄', '东升',                                   
'大峪', '城子', '东辛房', '大台', '王平', '永定', '龙泉', '潭柘寺', '军庄', '雁翅', '斋堂', '清水', '妙峰山',                                                  
'城关', '新镇', '向阳', '东风', '迎风', '星城', '良乡', '周口店', '琉璃河', '拱辰', '西潞', '阎村', '窦店', '石楼', '长阳', '长沟', '大石窝', '张坊', '十渡', '青龙湖', '韩村河', '霞云岭', '南窖', '佛子庄', '大安山', '史家营', '蒲洼',                                   
'中仓', '新华', '北苑', '玉桥', '潞源', '文景', '九棵树', '临河里', '杨庄', '潞邑', '永顺', '梨园', '宋庄', '张家湾', '漷县', '马驹桥', '西集', '台湖', '永乐店', '潞城', '于家务回族',                                          
'胜利', '光明', '仁和', '后沙峪', '天竺', '杨镇', '牛栏山', '南法信马坡', '石园', '空港', '双丰', '旺泉', '高丽营', '李桥', '李遂', '南彩', '北务', '大孙各庄', '张镇', '龙湾屯', '木林', '北小营', '北石槽', '赵全营',                                       
'城北', '南口', '马池口', '沙河', '城南', '东小口', '天通苑北', '天通苑南', '霍营', '回龙观', '龙泽园', '史各庄', '阳坊', '小汤山', '南邵', '崔村百善', '北七家', '兴寿', '流村', '十三陵', '延寿',                                          
'兴丰', '林校路', '清源', '亦庄', '黄村', '旧宫', '西红门', '瀛海', '观音寺', '天宫院', '高米店', '荣华', '博兴', '青云店', '采育', '安定', '礼贤榆垡', '庞各庄', '北臧村', '魏善庄', '长子营',                                          
'泉河', '龙山', '怀柔', '雁栖', '庙城', '北房', '杨宋', '桥梓', '怀北', '汤河口', '渤海', '九渡河', '琉璃庙', '宝山', '长哨营', '喇叭沟门',                                               
'滨河', '兴谷', '渔阳', '峪口', '马坊', '金海湖', '东高村', '山东庄', '南独乐河', '大华山', '夏各庄', '马昌营', '王辛庄', '大兴庄', '刘家店', '罗营', '黄松峪熊儿寨',                                              
'鼓楼', '果园', '檀营', '密云', '溪翁庄', '西田各庄', '十里堡', '河南寨', '巨各庄', '穆家峪', '太师屯', '高岭', '不老屯', '冯家峪', '古北口', '大城子', '东邵渠', '北庄新城子', '石城',                                            
'百泉', '香水园', '儒林', '延庆', '康庄', '八达岭', '永宁', '旧县', '张山营', '四海', '千家店', '沈家营', '大榆树', '井庄', '大庄科', '刘斌堡', '香营', '珍珠泉']
xinguan_list=['新增', '疫情', '轨迹', '感染', '阳性', '无症状', '新冠', '奥密克戎', '德尔塔', '行程', '核酸', '风险', '防范', '管控', '传播', '隔离', '密接', '确诊', '封控', '病例']

In [395]:
# AmapAPI return full address & LatLng
def call_amapAPI(address):
    url = 'http://restapi.amap.com/v3/geocode/geo?parameters'
    KEY = '460c7053ffe6fd4767354484f72f30df'
    adict={}
    CITY = '北京市'
    ADDRESS = address
    
    params = {
        'city': CITY,
        'address': ADDRESS, 
        'key': KEY,
        'output': 'json'}
    j = json.loads(requests.get(url, params).content)
    adict['formatted_address']= j['geocodes'][0]['formatted_address']
    adict['output_location']=[Decimal(j['geocodes'][0]['location'].split(',')[1]),
                              Decimal(j['geocodes'][0]['location'].split(',')[0])]
    return adict

In [396]:
# call-amaAPI result
adict=call_amapAPI('博雅西园')

In [397]:
f = folium.Figure(width=1000, height=700)
# define the city map
city_map = folium.Map(tiles='https://webrd02.is.autonavi.com/appmaptile?lang=zh_en&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
           attr='高德-中英文对照',
    location=[39.93, 116.40], 
#     location=[40.002694, 116.322373], #tsing hua
                      zoom_start=11,
                      min_zoom=10,
                      max_zoom=18,
                     control_scale=True,
                      no_touch=False).add_to(f)

# add clickLatLng
city_map.add_child(folium.LatLngPopup())

#add clickMarker
# city_map.add_child(folium.ClickForMarker(popup='Waypoint'))
for addr in addr_all:
    try:
        adict=call_amapAPI(addr)
        #add geo marker
        folium.Marker(
            location =adict['output_location'],
            popup=parse_zhch(adict['formatted_address']), 
            tooltip=parse_zhch('确诊轨迹'),
                icon=folium.Icon(color='red' , prefix='fa' , icon='exclamation-triangle')
        ).add_to(city_map)
    except:
        pass

# display city map
city_map


In [ ]:
# #禁止图片和css加载
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.managed_default_content_settings.images": 2}
# chrome_options.add_experimental_option("prefs", prefs)

# option = ChromeOptions()
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# # 如果想加载图片，就把下面第二句话改第一句话，删掉上面的“禁止图片和css加载”部分
# # wb = Chrome(options=option)
# driver = webdriver.Chrome("/Users/jialuwang/Downloads/chromedriver",
#                           options=chrome_options) 




# # LoginWeibo

# def loginWeibo(username, password): 
#     driver.get('https://passport.weibo.cn/signin/login') 
#     time.sleep(1) 
#     driver.find_element_by_id("loginName").send_keys(username) 
#     driver.find_element_by_id("loginPassword").send_keys(password) 
#     time.sleep(1) 
#     driver.find_element_by_id("loginAction").click() 
#     time.sleep(2)
#     driver.find_element_by_id('messageCheck').click()
#     time.sleep(10)
#     driver.find_element_by_id('message_sms_login').click()
#     yanzheng=input("输入六位验证码：")
#     for i in range(0,6):
#         driver.find_element_by_xpath('//div[@class="num clearfix"]/input[{}]'.format(i+1)).send_keys("yanzheng[i]")
#     time.sleep(5)
#     cookies = driver.get_cookies()
#     cookie = [item["name"] + "=" + item["value"] for item in cookies]
#     cookiestr = ';'.join(item for item in cookie)
#     return cookiestr
#     #driver.close() 
# driveLogIn = loginWeibo(username_wb, password_wb)
# driveLogIn

In [ ]:
# # TESTING------------
# def get_page():
#     query = {
#         'type':'uid',
#         'value':'2830678474',
#         'containerid':'1076031644948230',
#         'since_id':''
#     }
#     url = url_wb + urlencode(query)
#     try:
#         response = requests.get(url,headers=headers,verify=False)
#         if response.status_code == 200:
#             return response.json()
#     except requests.ConnectionError as e:
#             print('Get First Page Error',e.args)
            
# wb_json=get_page()


# i=1
# info_list=[]
# for card in wb_json['data']['cards']:
#     mblog=card['mblog']
#     mid = mblog['mid']
#     time=mblog['created_at']
#     reposts_count = mblog['reposts_count']
#     comments_count = mblog['comments_count']
#     attitudes_count = mblog['attitudes_count']
#     title=mblog['page_info']['page_title']
#     content1=mblog['page_info']['content1']
    
#     try:
#         content2=mblog['page_info']['content2']
#     except:
#         content2=''
        
#     isLongText = mblog['isLongText']
    
#     text = pq(mblog['text']).text()

#     if isLongText:
#         print('looong')
#         text_id = mblog['id']
#         text_href = 'http://m.weibo.cn/statuses/extend?id=%s'%text_id
#         text_res = requests.get(text_href,headers=headers,verify=False)
#         text_json=text_res.json()
#         text = pq(text_json['data']['longTextContent']).text()
# #         print(pq(text).text())          
#     info = (mid, time, reposts_count, comments_count, attitudes_count,title,content1,content2,text)
#     info_list.append(info)
# #     print(card)
#     print('-----------------',i,'-------------------')
#     i+=1

In [ ]:
# import cpca
# # location_str=bdf['text']
# # location_str=location_str.replace("\n",'')
# add_df=pd.DataFrame()
# for loc in bdf['text']:
#     try:
#         cdf = cpca.transform_text_with_addrs(loc)
#         cdf = cdf[(cdf['省']=='北京市')&(cdf['市'].notnull())]
#     except Exception as e:
# #         traceback.print_exc()
#         continue
#     add_df=add_df.append(cdf)
    
# add_df
    